In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import act
import glob
import matplotlib as mpl
import pyart

In [ ]:
mpl_10 = glob.glob('/home/jhemedinger/dev_act/act-analyses/data/sgp30smplcmask1zwangC1.c1/sgp30smplcmask1zwangC1.c1.2010*')
ceil_10 = glob.glob('/home/jhemedinger/dev_act/act-analyses/data/sgpceilC1.b1.2010/*')
mpl_10.sort()
ceil_10.sort()

In [ ]:
mpl_10.pop(0)
mpl_10.pop(10)
mpl_10.pop(42)
mpl_10.pop(42)
mpl_10.pop(100)
mpl_10.pop(117)
mpl_10.pop(128)
mpl_10.pop(128)
mpl_10.pop(142)
mpl_10.pop(148)
mpl_10.pop(148)
mpl_10.pop(171)
mpl_10.pop(171)
mpl_10.pop(171)
mpl_10.pop(178)
mpl_10.pop(181)
mpl_10.pop(181)
mpl_10.pop(305)
mpl_10.pop(305)

In [ ]:
ceil_10.pop(0)
ceil_10.pop(10)
ceil_10.pop(42)
ceil_10.pop(42)
ceil_10.pop(100)
ceil_10.pop(117)
ceil_10.pop(128)
ceil_10.pop(128)
ceil_10.pop(142)
ceil_10.pop(148)
ceil_10.pop(148)
ceil_10.pop(171)
ceil_10.pop(171)
ceil_10.pop(171)
ceil_10.pop(178)
ceil_10.pop(181)
ceil_10.pop(181)
ceil_10.pop(305)
ceil_10.pop(305)

In [ ]:
%%time
cbh_mpl_10 = []
time_mpl_10 = []
cbh_ceil_10 = []
time_ceil_10 = []

for mpl in mpl_10:
    mpl_ds = act.io.armfiles.read_netcdf(mpl)
    new_mpl = mpl_ds.resample(time='1min').interpolate('linear')
#     time_mpl_10.append(new_mpl.time.copy().shape)
    nmpl_cbh = np.ma.array(new_mpl.cloud_base.values.copy(), dtype=np.float32)
    nmpl_time = np.ma.array(new_mpl.time.values.copy(), dtype=np.datetime64)
    for i in range(len(nmpl_cbh)):
        cbh_mpl_10.append(nmpl_cbh[i])
        time_mpl_10.append(nmpl_time[i])

for ceil in ceil_10:
    ceil_ds = act.io.armfiles.read_netcdf(ceil)
    new_ceil = ceil_ds.resample(time='1min').interpolate('linear')
#     time_ceil_10.append(new_ceil.time.copy().shape)
    nceil_cbh = np.ma.array(new_ceil.first_cbh.values.copy(), dtype=np.float32)
    nceil_time = np.ma.array(new_ceil.time.values.copy(), dtype=np.datetime64)
    for i in range(len(nceil_cbh)):
        cbh_ceil_10.append(nceil_cbh[i])
        time_ceil_10.append(nceil_time[i])

In [ ]:
mm_10 = np.ma.array(cbh_mpl_10, dtype=np.float32)
cc_10 = np.ma.array(cbh_ceil_10, dtype=np.float32)
tm_10 = np.ma.array(time_mpl_10, dtype=np.datetime64)
tc_10 = np.ma.array(time_ceil_10, dtype=np.datetime64)

mcbh_10 = mm_10[np.ma.where((cc_10 >= 500.) & (cc_10 <= 7620.))[0]]
ccbh_10 = cc_10[np.ma.where((cc_10 >= 500.) & (cc_10 <= 7620.))[0]]
mtime_10 = tm_10[np.ma.where((cc_10 >= 500.) & (cc_10 <= 7620.))[0]]
ctime_10 = tc_10[np.ma.where((cc_10 >= 500.) & (cc_10 <= 7620.))[0]]

a_10 = ccbh_10[np.ma.where((mcbh_10 >= .5) & (mcbh_10 <= 7.620))[0]]
b_10 = mcbh_10[np.ma.where((mcbh_10 >= .5) & (mcbh_10 <= 7.620))[0]]
c_10 = ctime_10[np.ma.where((mcbh_10 >= .5) & (mcbh_10 <= 7.620))[0]]
d_10 = mtime_10[np.ma.where((mcbh_10 >= .5) & (mcbh_10 <= 7.620))[0]]

In [ ]:
print(a_10.min())
print(a_10.max())
print(b_10.min()*1000)
print(b_10.max()*1000)

In [ ]:
print(mcbh_10.shape)
print(ccbh_10.shape)
print(mtime_10.shape)
print(ctime_10.shape)
print(a_10.shape)
print(b_10.shape)
print(c_10.shape)
print(d_10.shape)

In [ ]:
f = (a_10/1000 - b_10)

In [ ]:
fig = plt.figure(figsize=[50,12])
p1 = plt.scatter(c_10, f)
#plt.xlim('20090501', '20090601')
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.title('2010 Ceil MPL Height Difference vs Time', fontsize=30)
plt.xlabel('TIME (UTC)', fontsize=25)
plt.ylabel('HEIGHT (m)', fontsize=25)
#plt.ylim(4000, 7262)
#plt.savefig('/home/jhemedinger/dev_act/act-analyses/ceil_mpl_hght_diff_200905.png')
plt.show()
plt.close()

In [ ]:
print(f.mean()*1000)
print(f.std()*1000)

In [ ]:
fig = plt.figure(figsize=[25,25])
plt.scatter(a_10/1000, b_10)
plt.plot(range(0,25), range(0,25), 'k', label='1:1')
plt.plot(np.unique(a_10/1000), np.poly1d(np.polyfit(a_10/1000, b_10, 1))(np.unique(a_10/1000)), 'k--', label='Best Fit Line')
plt.plot(0, 0, 'w', label= ('Mean Difference: ' + '%6.4f km' %(f.mean())))
plt.plot(0, 0, 'w', label= ('STDEV Difference: ' + '%6.4f km' %(f.std())))
plt.xlim(.5, 8)
plt.ylim(.5, 8)
plt.xticks(np.arange(0,8), fontsize=25)
plt.yticks(range(0,8), fontsize=25)
plt.ylabel('MPL Heights (km)', fontsize=30)
plt.xlabel('CEIL Heights (km)', fontsize=30)
plt.title('2010 Ceil vs. MPL Heights', fontsize=30)
plt.legend(loc=4, labelspacing=1, ncol=4, fontsize=18)
plt.savefig('/home/jhemedinger/dev_act/act-analyses/ceil_v_mpl_hghts_2010.png')
plt.show()
plt.close()

In [ ]:
norm = mpl.colors.LogNorm()

In [ ]:
xedges = np.arange(.5, 7.620, 0.25)
yedges = np.arange(.5, 7.620, 0.25)

In [ ]:
fig = plt.figure(figsize=[25,25])
plt.hist2d(a_10/1000, b_10, bins=(xedges, yedges),
           cmap=pyart.graph.cm_colorblind.HomeyerRainbow,
           norm=norm)
plt.plot(range(0,25), range(0,25), 'k', label='1:1')
plt.plot(np.unique(a_10/1000), np.poly1d(np.polyfit(a_10/1000, b_10, 1))(np.unique(a_10/1000)), 'k--', label='Best Fit Line')
plt.plot(0, 0, 'w', label= ('Mean Difference: ' + '%6.4f km' %(f.mean())))
plt.plot(0, 0, 'w', label= ('STDEV Difference: ' + '%6.4f km' %(f.std())))
plt.xlim(.5, 8)
plt.ylim(.5, 8)
plt.xticks(np.arange(0,8), fontsize=25)
plt.yticks(range(0,8), fontsize=25)
plt.ylabel('MPL Heights (km)', fontsize=30)
plt.xlabel('CEIL Heights (km)', fontsize=30)
plt.title('2010 Ceil vs. MPL Heights', fontsize=30)
plt.legend(loc=4, labelspacing=1, ncol=4, fontsize=18)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=25)
plt.savefig('/home/jhemedinger/dev_act/act-analyses/images/ceil_v_mpl_hghts_2d_2010.png')
plt.show()
plt.close()